In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv')

In [ ]:
df.head()

In [ ]:
import pandas as pd 
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize,stem
from wordcloud import WordCloud, STOPWORDS

In [ ]:
df.drop('hindi_sentence',axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df['source']= le.fit_transform(df['source'])

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
df['english_sentence'].str.len().head()

In [ ]:
df['lower_english_sentence'] = df['english_sentence'].str.lower()

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
def nlp_preprocesser(row):
    sentence = row.lower_english_sentence
    #convert all characters to lowercase
    lowered = sentence
    tok = tokenize.word_tokenize(lowered)
 
    #lemmatizing & stemming
    lemmatizer = stem.WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(i) for i in tok if i not in STOPWORDS]
    stemmer = stem.PorterStemmer()
    stems = [stemmer.stem(i) for i in lem if i not in STOPWORDS]
 
    #remove non-alphabetical characters like '(', '.' or '!'
    alphas = [i for i in stems if i.isalpha() and (i not in STOPWORDS)]
    return " ".join(alphas)

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')

In [ ]:
df['new_english_sentence'] = df.apply(nlp_preprocesser, axis=1)

In [ ]:
df.isna().apply(pd.value_counts).T

In [ ]:
df.head()

In [ ]:
 wordcloud = WordCloud(width = 1500, height = 800, random_state=0, background_color='black', colormap='rainbow', \
                      min_font_size=5, max_words=300, collocations=False, stopwords = STOPWORDS).generate(" ".join(df['new_english_sentence'].values))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
 feature_df = pd.DataFrame()
for i in [1,2,3]:
    vec_tfidf = TfidfVectorizer(max_features=10, norm='l2', stop_words='english', lowercase=True, use_idf=True, ngram_range=(i,i))
    X = vec_tfidf.fit_transform(df['new_english_sentence']).toarray()
    tfs = pd.DataFrame(X, columns=["TFIDF_" + n for n in vec_tfidf.get_feature_names()])
    feature_df = pd.concat([feature_df, tfs], axis=1)
feature_df = pd.concat([df, feature_df], axis=1)

In [ ]:
feature_df.head()

In [ ]:
feature_df.dropna(axis=0, inplace=True)

In [ ]:
X=feature_df.drop(['english_sentence','lower_english_sentence','new_english_sentence','source'],axis=1)

In [ ]:
y=feature_df[['source']]

In [ ]:
df.shape

In [ ]:
feature_df.shape

In [ ]:
feature_df.isna().apply(pd.value_counts).T

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
lr= LogisticRegression(multi_class='multinomial',solver='lbfgs')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
lr_pred = lr.predict(X_test)

In [ ]:
lr_pred

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(lr_pred,y_test)

In [ ]:
lr.score(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
accuracy_score(rf_pred,y_test)

In [ ]:
rf.score(X_train,y_train)

Model accuracy is too low. We'll use LSTM to overcome this

In [ ]:
import pandas as pd
import numpy as np
import nltk
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('../input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv')

In [ ]:
df.head()

In [ ]:
df['source'] = le.fit_transform(df['source'])

In [ ]:
X = df[['english_sentence','hindi_sentence']]
y = df['source']

In [ ]:
X.head()

In [ ]:
y = y.values.reshape(-1,1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
#Tokenization
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
model1 = Sequential()
model1.add(Embedding(max_words,128))
model1.add(LSTM(128,dropout=0.2,input_shape=[max_len]))
model1.add(Dense(1,activation='sigmoid'))

In [ ]:
model1.summary()
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model1.fit(sequences_matrix,y_train,batch_size=128,epochs=10,validation_split=0.3,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])